# How do Dictionaries Work? 

In [3]:
niceties = {"greeting" : "Hello", "departing" : "Goodbye", "requesting" : "please" }

## 1. Load Factor

In [27]:
import sys

d = {}
prev_size = sys.getsizeof(d)

for i in range(1000):
    d[i] = i
    bytes_of_data = bytes_of_data + sys.getsizeof(d[i]) + sys.getsizeof(i)
        
    new_size = sys.getsizeof(d)
    if new_size != prev_size:
        print(f"Resize at {i} entries: size of underlying array changed from {prev_size} → {new_size} bytes")
        prev_size = new_size

Resize at 0 entries: size of underlying array changed from 64 → 232 bytes
Resize at 5 entries: size of underlying array changed from 232 → 360 bytes
Resize at 10 entries: size of underlying array changed from 360 → 640 bytes
Resize at 21 entries: size of underlying array changed from 640 → 1176 bytes
Resize at 42 entries: size of underlying array changed from 1176 → 2272 bytes
Resize at 85 entries: size of underlying array changed from 2272 → 4696 bytes
Resize at 170 entries: size of underlying array changed from 4696 → 9312 bytes
Resize at 341 entries: size of underlying array changed from 9312 → 18520 bytes
Resize at 682 entries: size of underlying array changed from 18520 → 36960 bytes


We face a tradeoff with selecting the load factor and resize strategy because the lower the load factor and the more we resize by, the more space the dict takes up in memory that maybe it won't need for actual entries. 

But a higher load factor means more dealing with collisions before a resize, which can make reads and writes slower, and the less we resize by, the more often we have to do the resize operation...which makes writes slower also, as the whole dict now has to be transferred to a larger table before the write that triggered the resize is complete.

## 2. Hashing the Keys

A good hash function is: 

1. Deterministic
2. Uniform
3. Fast

# [--------] [--------] [--------] [--------] [--------] [--------] [--------] [--------] 

In [11]:
hash("greeting")

2901722735092097552

In [12]:
hash("greeting") % 8

0

# ["greeting"] [--------]  [--------] [--------] [--------] [--------] [--------] [--------] 

In [13]:
hash("departing") % 8

4

# ["greeting"] [-------] [-------] [-------] ["departing"] [-------] [-------] [-------] 

In [14]:
hash("requesting") % 8

5

# ["greeting"] [------] [------] [------] ["departing"] ["requesting"] [------] [------] 

In [18]:
hash("thanking") % 8 

0

## 3. Separate Chaining vs Open Addressing and Probing

This would be separate chaining. This is what Java, Go, C#, and C++ does.

# ["greeting"][-----][-----] [-----] ["departing"] ["requesting"] [-----] [-----] 
# ---- v ---
# ["thanking"] 

Open addressing with probing is what Python, Ruby, and Rust all do.

Now, the below demonstrates open addressing with straight linear probing.


# ["greeting"] ["thanking"] [-----] [-----] ["departing"] ["requesting"] [-----] [-----] 

In [19]:
hash("apologizing") % 8

4

Now, linear probing is not exactly what Python does, because linear probing tends to result in a bunch of overflowing buckets right next to each other, which, like separate chaining, approaches O(n) efficiency the worse it gets. Instead, when a Python dictionary encounters a hash collision, it uses the original hash value to determine the next index to visit. 

Now, it _doesn't_ exactly hash the original hashed value (though you _could_). It might instead be something like:

In [24]:
perturb = hash("apologizing") # This is NOT exactly how the perturb is calculated—the details are a bit more complicated—but it does _use_ the original hash
current_index = hash("apologizing") % 8
new_value_including_original_hash = current_index * 5 + perturb + 1 
new_value_including_original_hash % 8

1

# ["greeting"] ["thanking"] [--] [--] ["departing"] ["requesting"] ["apologizing"] [--] 

In [20]:
hash("purchasing") % 8

7

## ["greeting"] ["thanking"] [--] [--] ["departing"] ["requesting"] ["apologizing"] ["purchasing"] 

Here, more than 2/3rds of the entries are full, and this is CPython's default load factor. 

At this point, Python increases the size of the array. The way it does this is....

1. It multiplies the number of FILLED entries by 3 (so in this case, 6 * 3 is 18)
2. It rounds up to the nearest power of 2 (it does this because...sigh....okay it _does this_ because bitmasking and bitshifting are much faster operations for doing multiplication and division operations in C than actual multiplication and division are, but you need a binary digit to use them. It makes a big enough difference to the speed of Python to be worth the extra space allocated this way.) In this case that's gonna be 32 entries now, because 2 ** 4 is only sixteen and the next one up is 2 ** 5. 




# 4. Tombstones

A _tombstone_ is a value that we use to replace a deleted value. Right now in our example above, if we delete "greeting" and leave that slot empty...

# [--] ["thanking"] [--] [--] ["departing"] ["requesting"] ["apologizing"] [--] 

...and then we try to find "thanking", we do...

In [26]:
hash("thanking") % 8

0

and go to the first spot in the table, and see _nothing there_. The way we find values in the table is we _walk from their original hash index along the probing sequence until we either find them, or we find an empty slot...at which point the language assumes it's _not there_ (maybe it has been deleted, for example). But this one HAS NOT been deleted. It's just that the probing sequence has a deleted element. SO instead we do...

# [TOMBSTONE] ["thanking"] [--] [--] ["departing"] ["requesting"] ["apologizing"] [--] 

This leaves the probe chain intact for us to follow to find "thanking" after all :)

Note that tombstones are a detail of the underlying implementation of the open addressing protocol because there's no structural difference between an entry that required probing to be placed and an entry that didn't. If you use separate chaining and you store the entries in linked lists, you can delete a thing from a linked list and stitch the two sides of the list back together (in fact, this is what linked lists are for) and it'll work. Your one issue there would be deleting the FIRST item in the linked list; rather than then leave that spot empty, since it's the entry point, you either have to move the formerly-second-now-first item into that spot, or you have to do a tombstone there. As with other tombstones, you can remove it when you resize the underlying table and then reenter all the entries into the new table. 

"Tombstone" is a word we tend to use in programming to describe any strategy in which we do something _other than deletion_ upon the end user asking to delete something, usually to make the program run the way the user expects after the deletion. You have to do something like this in your homework to handle a user deleting a key in a transaction because you cannot tell the main data store to remove a key just by not having that key in the update store. You have to devise some way to indicate that a column is supposed to be deleted, either by making a list of column names to delete upon commission or setting the value of the key in the update table to a sentinel value like "DELETEME123." This second strategy, while not requiring a new data structure like the first strategy, runs the risk of colliding with a user _actually_ wanting to set a column to whatever the sentinel value is, but you can choose a sentinel value that's really, really unlikely (for example, given that columns are represented with lists, a string like "DELETEME123" is very unlikely to be _intentionally_ set by a user). Though this strategy looks primitive, it was ACTUALLY part of the fabric of the original implementation of Facebook's Cassandra project for darge data storage.